# Image Moderation

In this tutorial, we will show how you can use Amazon Rekognition and Amazon Comprehend to quickly identify unsafe/inappropriate content in your image libraries.

Below is the list of AWS services that we're going to use in this notebook. 
- [Amazon Rekognition](https://aws.amazon.com/rekognition/) makes it easy to add image and video analysis to your applications.
- [Amazon Comprehend](https://aws.amazon.com/comprehend/) uses natural language processing (NLP) to extract insights about the content of documents.
- [Amazon Simple Storage Service (Amazon S3)](https://aws.amazon.com/s3/) is an object storage service that offers industry-leading scalability, data availability, security, and performance.
- [Amazon SageMaker](https://aws.amazon.com/sagemaker/) allows you to build, train, and deploy machine learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.

![image-moderation-with-text-arc](../images/image-moderation-with-text.png)

Follow the steps in this notebook to understand how you can identify inappropriate content and PII information in an image. For more in depth instructions, Please visit [this link](https://docs.aws.amazon.com/rekognition/latest/dg/procedure-moderate-images.html)

- [Step 1: Setup Notebook](#step1)
- [Step 2: Moderate image using Rekognition image moderation API](#step2)
- [Step 3: Moderate text in image using Comprehend PII detection API](#step3)
- [Step 4: Clean up](#step4)

# Step 1: Setup Notebook <a id="step1"></a>
Run the below cell to install/update Python dependencies if you run the lab using a local IDE. It is optional if you use a SageMaker Studio Juypter Notebook, which already includes the dependencies in the kernel. 

In [ ]:
%pip install -qU pip
%pip install boto3 -qU

In [ ]:
import boto3
import sagemaker

# variables
data_bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()
region = boto3.session.Session().region_name

s3 = boto3.client('s3', region_name=region)
rekognition = boto3.client('rekognition', region_name=region)
comprehend = boto3.client('comprehend', region_name=region)


print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

For this lab, we will use a sample image stored in this repo: ../datasets/yoga_swimwear_lighttext.jpg.

The image contains a lady in a bikini and Rekognition Image Moderation will label it as "Suggestive". Also as the image contains phone number, we can use the combination of Amazon Rekognition and Amazon Comprehend together to detect and label the phone number as PII (Personal Identification Information).

![image-moderation-hitl-arc](../datasets/yoga_swimwear_lighttext.jpg)

Now, let's upload the image to the default S3 bucket for Rekognition to access.

In [ ]:
s3_key = 'content-moderation-im/image-moderation/yoga_swimwear_lighttext.jpg'
s3.upload_file('../datasets/yoga_swimwear_lighttext.jpg', data_bucket, s3_key)

# Step 2: Moderate image <a id="step2"></a>
In this step, we will call Amazon Rekognition [Content Moderation API](https://docs.aws.amazon.com/rekognition/latest/dg/procedure-moderate-images.html) to detect inappropriate content in the image. 

In [ ]:
detectModerationLabelsResponse = rekognition.detect_moderation_labels(
    Image={
       'S3Object': {
           'Bucket': data_bucket,
           'Name': s3_key,
       }
    }
)
detectModerationLabelsResponse

Note the response from a call to DetectModerationLabels:

- ModerationLabels – The example shows a list of labels for inappropriate or offensive content found in the image. The list includes the top-level label and each second-level label that are detected in the image. Please see [Amazon Rekognition doucmentation](https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html) for the complete list of supported top/second level labels.

- Name/ParentName – Each label has a name, an estimation of the confidence that Amazon Rekognition has that the label is accurate, and the name of its parent label. The parent name for a top-level label is "".

- Confidence – Each label has a confidence value between 0 and 100 that indicates the percentage confidence that Amazon Rekognition has that the label is correct. You specify the [minimal confidence level](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectModerationLabels.html#rekognition-DetectModerationLabels-request-MinConfidence) for a label to be returned in the response in the API operation request.

In [ ]:
for label in detectModerationLabelsResponse["ModerationLabels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
    print("  - Parent: {}".format(label["ParentName"]))

As we can see in the Rekognition response, the Image Moderation API labeled the image in 3 categories with confidence scores:
- Top level category: **Suggestive** with a confidence score > 98%
- Second level category: **Female Swimwear Or Underwear** with a confidence score > 98%
- Second level category: **Revealing Clothes** with a confidence score ~ 68% 

# Step 3: Moderate text in image <a id="step3"></a>
Now, let's see how to extract text in an image and detect if it contains sensitive information. In this step. we are going to use Rekognition API to [detect text in image](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectText.html) and pass it on to Amazon Comprehend API to [detect PII](https://docs.aws.amazon.com/comprehend/latest/APIReference/API_DetectPiiEntities.html).

In [ ]:
detectTextResponse = rekognition.detect_text(
    Image={
       'S3Object': {
           'Bucket': data_bucket,
           'Name': s3_key,
       }
    }
)

detected_text_list=[]
textDetections=detectTextResponse['TextDetections']
for text in textDetections:
    if text['Type'] == 'LINE':
        print ('Detected text: ' + text['DetectedText'])
        detected_text_list.append(text['DetectedText'])
    
    
for text in detected_text_list:
    response=comprehend.detect_pii_entities(Text=text, LanguageCode='en')
    if len(response['Entities']) > 0: 
        print("Detected PII entity is: " + text[response['Entities'][0]['BeginOffset']:response['Entities'][0]['EndOffset']])
        print("PII type is: " + response['Entities'][0]['Type'])
        print("Confidence score is: " + str(response['Entities'][0]['Score']))
        print()

# Step 4: Clean up <a id="step4"></a>

Remove the resources that may incur the cost

In [ ]:
# remove the image from s3 bucket
response = s3.delete_object(Bucket=data_bucket, Key=s3_key)
print(response)

# Conclusion
In this lab, we moderated an image (containing text) using Amazon Rekognition and Amazon Comprehend.